In [ ]:
!pip install --upgrade openpyxl

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

#Catalogo

catalogo de productos

In [ ]:
ab=pd.read_excel("/content/drive/Shareddrives/Catalogos_SO/Prod_Holding.xlsx",engine="openpyxl")
ab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2245 entries, 0 to 2244
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   key              2244 non-null   object
 1   Site             2245 non-null   object
 2   Holding          2245 non-null   object
 3   identificador    2132 non-null   object
 4   descripcion      2245 non-null   object
 5   CODIGO           2006 non-null   object
 6   codigo_anterior  53 non-null     object
 7   EAN              671 non-null    object
 8   Factor_Caja      2202 non-null   object
dtypes: object(9)
memory usage: 158.0+ KB


In [ ]:
# seleccion de columnas por index
ab=ab.iloc[:,[1,2,3,5]]

# filtrado sobre el catalogo
ab = ab[ab['Site'] == 'MTY']
ab = ab[ab['Holding'] == 'DUERO']

# selccionar columnas por index
ab= ab.iloc[:,[2,3]]

# renombrar las columnas
ab= ab.rename(columns={'identificador': "CLAVE"})
ab= ab.rename(columns={'CODIGO': "AB actual"})

In [ ]:
ab.head()

,CLAVE,AB actual
226,33798,AB17335636
227,23544,AB15090074
228,51931,AB10332659
229,9983,AB14101664
230,46584,AB10331560


catalogo de tiendas

In [ ]:
tda= pd.read_excel("/content/drive/Shareddrives/Catalogos_SO/Tda Duero.xlsx", engine="openpyxl") # Explicitly use the openpyxl engine

tda= tda.iloc[:,[1,2]] #seleccionar columnas de interes

tda['Nombre de Tienda'] = tda['Nombre de Tienda'].str.replace('Duero', '')  #remplazar nombres

tda= tda.rename(columns={'Código Punto de Venta Cadena': "id"})  #renombrar una columna

tda = tda[["id", "Nombre de Tienda"]]  #reordenar las columnas

tda['Nombre de Tienda']=tda['Nombre de Tienda'].str.strip() #eliminar espacios en blanco en toda posicion

#tda['Nombre de Tienda'] = tda['Nombre de Tienda'].str.lower()  #minisculas

tda.head()

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,id,Nombre de Tienda
0,C001,Bodega Zamora
1,C002,Bodega Uruapan
2,C003,Bodega Colima
3,C004,Bodega Morelia
4,C005,Bodega Lazaro Cardenas


#SEMANAL

### Venta

In [ ]:
#omitir notacion cientifica
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
sem = pd.read_excel("/content/drive/MyDrive/Sell Out/Duero/Semanal/vta/2024/24.41.xlsx", engine='openpyxl') # Explicitly use the openpyxl engine
sem.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,11162.70
0,SEMANA 41,NaN,NaN,NaN,NaN,NaN,Vta Cajas
1,Centro,Nombre,Cedis Primario,Nombre Cedis,Material,Año natural/Semana,41.2024
2,C001,BODEGA ZAMORA,C001,B. ZAMORA,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,22
3,C001,BODEGA ZAMORA,C001,B. ZAMORA,9983,SVT PREMIER 12/450PZ,20
4,C001,BODEGA ZAMORA,C001,B. ZAMORA,23544,HIG PREMIER 400HD 12/4ROLL,115


Ajustar columnas

In [ ]:
sem = sem[2:]  #eliminar las primeras dos filas

sem = sem.iloc[:, [0,1,4,5,6]]  #seleccionar columnas por index

#Renaming columns by index (el nuevo dataset tiene nuevas columnas ordenadas)
sem.columns.values[0] = "tda"
sem.columns.values[1] = "nom tda"
sem.columns.values[2] = "prod"
sem.columns.values[3] = "descripcion"
sem.columns.values[4] = "cjas"

sem.head()

,tda,nom tda,prod,descripcion,cjas
2,C001,BODEGA ZAMORA,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,22
3,C001,BODEGA ZAMORA,9983,SVT PREMIER 12/450PZ,20
4,C001,BODEGA ZAMORA,23544,HIG PREMIER 400HD 12/4ROLL,115
5,C001,BODEGA ZAMORA,28817,HIG ELITE COLOR 287HD 10/4ROLL,1
6,C001,BODEGA ZAMORA,33798,HIG ELITE BIG ROLL 550HD 8/6PZ,313


Revisar valores del dataset

In [ ]:
sem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1550 entries, 2 to 1551
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tda          1550 non-null   object
 1   nom tda      1550 non-null   object
 2   prod         1550 non-null   object
 3   descripcion  1550 non-null   object
 4   cjas         1550 non-null   object
dtypes: object(5)
memory usage: 60.7+ KB


In [ ]:
print("Missing values distribution: ")
print(sem.isnull().mean())
print("")
sem.shape

Missing values distribution: 
tda           0.00
nom tda       0.00
prod          0.00
descripcion   0.00
cjas          0.00
dtype: float64



(1550, 5)

Transformar el tipo de datos del dataset

In [ ]:
sem["cjas"] = sem["cjas"].astype(float)

Valores negativos a positivos

In [ ]:
sem['cjas'] = np.where(sem['cjas'] < 0, 0, sem['cjas'])

seleccionar valores mayores a 0´s

In [ ]:
sem = sem[(sem['cjas']>0)]

Eliminar todo los espacios en blanco del dataset final

In [ ]:
sem = sem.replace(r'\s+', ' ', regex=True) # espacios en blanco
sem.head()

<ipython-input-58-9f2cb8d3461e>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sem = sem.replace(r'\s+', ' ', regex=True) # espacios en blanco


,tda,nom tda,prod,descripcion,cjas
2,C001,BODEGA ZAMORA,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,22.00
3,C001,BODEGA ZAMORA,9983,SVT PREMIER 12/450PZ,20.00
4,C001,BODEGA ZAMORA,23544,HIG PREMIER 400HD 12/4ROLL,115.00
5,C001,BODEGA ZAMORA,28817,HIG ELITE COLOR 287HD 10/4ROLL,1.00
6,C001,BODEGA ZAMORA,33798,HIG ELITE BIG ROLL 550HD 8/6PZ,313.00


## Inventarios

In [ ]:
stocks = pd.read_csv("/content/drive/MyDrive/Sell Out/Duero/Semanal/invt/2024/24_41.csv", sep='|', encoding='utf8')

stocks=stocks.replace(r'\s+', ' ', regex=True)

stocks.head()

,Info_del_Mes,Centro,Des_Centro,Prov,Des_Prov,Material,Des_Material,Marca,Des_Marca,ABC,Cen.Primario,Des.Cen.Primario,Familia,Des_Familia,Inv_Umb,Inv_Cja
0,202410,C001,BODEGA ZAMORA,100998,ABSORMEX CMPC TISSUE SA DE CV,9742,PÑ BABYSEC CLASICO CHICO 4/40PZ,3569,BABYSEC,NaN,C352,CEDIS SAN ANGEL,2,ABARROTES NO COMESTIBLES,0.00,0.00
1,202410,C001,BODEGA ZAMORA,100998,ABSORMEX CMPC TISSUE SA DE CV,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,2977,ELITE,B,C352,CEDIS SAN ANGEL,2,ABARROTES NO COMESTIBLES,220.00,11.00
2,202410,C001,BODEGA ZAMORA,100998,ABSORMEX CMPC TISSUE SA DE CV,9983,SVT PREMIER 12/450PZ,3518,PREMIER,A,C352,CEDIS SAN ANGEL,2,ABARROTES NO COMESTIBLES,96.00,8.00
3,202410,C001,BODEGA ZAMORA,100998,ABSORMEX CMPC TISSUE SA DE CV,9984,SVT PREMIER 48/100PZ,3518,PREMIER,NaN,C352,CEDIS SAN ANGEL,2,ABARROTES NO COMESTIBLES,0.00,0.00
4,202410,C001,BODEGA ZAMORA,100998,ABSORMEX CMPC TISSUE SA DE CV,11233,HIG ELITE BIG ROLL 400HD 12/4ROLL,2977,ELITE,NaN,C352,CEDIS SAN ANGEL,2,ABARROTES NO COMESTIBLES,0.00,0.00


Delimitar el dataset

In [ ]:
stocks = stocks.iloc[:, [0,1,2,5,6,15]]  #seleccionar columnas por index

#Renaming columns by index (el nuevo dataset tiene nuevas columnas ordenadas)
stocks.columns.values[0] = "fecha"
stocks.columns.values[1] = "tda"
stocks.columns.values[2] = "nom tda"
stocks.columns.values[3] = "prod"
stocks.columns.values[4] = "descripcion"
stocks.columns.values[5] = "inv cajas"

stocks.head()

,fecha,tda,nom tda,prod,descripcion,inv cajas
0,202410,C001,BODEGA ZAMORA,9742,PÑ BABYSEC CLASICO CHICO 4/40PZ,0.00
1,202410,C001,BODEGA ZAMORA,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,11.00
2,202410,C001,BODEGA ZAMORA,9983,SVT PREMIER 12/450PZ,8.00
3,202410,C001,BODEGA ZAMORA,9984,SVT PREMIER 48/100PZ,0.00
4,202410,C001,BODEGA ZAMORA,11233,HIG ELITE BIG ROLL 400HD 12/4ROLL,0.00


In [ ]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11662 entries, 0 to 11661
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   fecha        11662 non-null  int64  
 1   tda          11662 non-null  object 
 2   nom tda      11662 non-null  object 
 3   prod         11662 non-null  int64  
 4   descripcion  11662 non-null  object 
 5   inv cajas    11662 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 546.8+ KB


Filtrar el dataset de acuerdo a la ultima fecha del inventario

In [ ]:
stocks=stocks.loc[stocks["fecha"] != 202409]  #filtrado de fecha -1mes anterior

stocks.drop(["fecha"],axis="columns",inplace=True)  #eliminar una columna

<ipython-input-62-51c540923e8f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks.drop(["fecha"],axis="columns",inplace=True)  #eliminar una columna


valores negativos

In [ ]:
stocks['inv cajas'] = pd.to_numeric(stocks['inv cajas'], errors='coerce') #convertir a Num por fuerza bruta

stocks['inv cajas'] = np.where(stocks['inv cajas'] < 0, 0, stocks['inv cajas'])

<ipython-input-63-63e2e3a46776>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks['inv cajas'] = pd.to_numeric(stocks['inv cajas'], errors='coerce') #convertir a Num por fuerza bruta
<ipython-input-63-63e2e3a46776>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks['inv cajas'] = np.where(stocks['inv cajas'] < 0, 0, stocks['inv cajas'])


valores nulos

In [ ]:
stocks = stocks[(stocks['inv cajas']>0)]

## Merged catalogos

Venta

In [ ]:
#productos
merged_sem = sem.merge(ab, left_on="prod", right_on="CLAVE", how="left")
merged_sem.head()

,tda,nom tda,prod,descripcion,cjas,CLAVE,AB actual
0,C001,BODEGA ZAMORA,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,22.00,9979,AB10335223
1,C001,BODEGA ZAMORA,9983,SVT PREMIER 12/450PZ,20.00,9983,AB14101664
2,C001,BODEGA ZAMORA,23544,HIG PREMIER 400HD 12/4ROLL,115.00,23544,AB15090074
3,C001,BODEGA ZAMORA,28817,HIG ELITE COLOR 287HD 10/4ROLL,1.00,28817,AB12335678
4,C001,BODEGA ZAMORA,33798,HIG ELITE BIG ROLL 550HD 8/6PZ,313.00,33798,AB17335636


In [ ]:
#tda
merged_sem = merged_sem.merge(tda, left_on="tda", right_on="id", how="left")
merged_sem.head()

,tda,nom tda,prod,descripcion,cjas,CLAVE,AB actual,id,Nombre de Tienda
0,C001,BODEGA ZAMORA,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,22.00,9979,AB10335223,C001,Bodega Zamora
1,C001,BODEGA ZAMORA,9983,SVT PREMIER 12/450PZ,20.00,9983,AB14101664,C001,Bodega Zamora
2,C001,BODEGA ZAMORA,23544,HIG PREMIER 400HD 12/4ROLL,115.00,23544,AB15090074,C001,Bodega Zamora
3,C001,BODEGA ZAMORA,28817,HIG ELITE COLOR 287HD 10/4ROLL,1.00,28817,AB12335678,C001,Bodega Zamora
4,C001,BODEGA ZAMORA,33798,HIG ELITE BIG ROLL 550HD 8/6PZ,313.00,33798,AB17335636,C001,Bodega Zamora


Revisar valores nulos en el dataset

In [ ]:
print(merged_sem.isnull().sum())

tda                 0
nom tda             0
prod                0
descripcion         0
cjas                0
CLAVE               0
AB actual           0
id                  0
Nombre de Tienda    0
dtype: int64


In [ ]:
merged_sem[merged_sem.isna().any(axis=1)]  #revisar registros con datos faltantes

#merged_stocks.dropna(inplace=True) #eliminar registros con datos faltantes

,tda,nom tda,prod,descripcion,cjas,CLAVE,AB actual,id,Nombre de Tienda


Inventarios

In [ ]:
#productos
merged_stocks = stocks.merge(ab, left_on="prod", right_on="CLAVE", how="left")
merged_stocks.head()

,tda,nom tda,prod,descripcion,inv cajas,CLAVE,AB actual
0,C001,BODEGA ZAMORA,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,11.00,9979,AB10335223
1,C001,BODEGA ZAMORA,9983,SVT PREMIER 12/450PZ,8.00,9983,AB14101664
2,C001,BODEGA ZAMORA,23544,HIG PREMIER 400HD 12/4ROLL,270.00,23544,AB15090074
3,C001,BODEGA ZAMORA,28817,HIG ELITE COLOR 287HD 10/4ROLL,2.00,28817,AB12335678
4,C001,BODEGA ZAMORA,33798,HIG ELITE BIG ROLL 550HD 8/6PZ,2888.38,33798,AB17335636


In [ ]:
#tda
merged_stocks = merged_stocks.merge(tda, left_on="tda", right_on="id", how="left")
merged_stocks.head()

,tda,nom tda,prod,descripcion,inv cajas,CLAVE,AB actual,id,Nombre de Tienda
0,C001,BODEGA ZAMORA,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,11.00,9979,AB10335223,C001,Bodega Zamora
1,C001,BODEGA ZAMORA,9983,SVT PREMIER 12/450PZ,8.00,9983,AB14101664,C001,Bodega Zamora
2,C001,BODEGA ZAMORA,23544,HIG PREMIER 400HD 12/4ROLL,270.00,23544,AB15090074,C001,Bodega Zamora
3,C001,BODEGA ZAMORA,28817,HIG ELITE COLOR 287HD 10/4ROLL,2.00,28817,AB12335678,C001,Bodega Zamora
4,C001,BODEGA ZAMORA,33798,HIG ELITE BIG ROLL 550HD 8/6PZ,2888.38,33798,AB17335636,C001,Bodega Zamora


Revisar valores nulos en el dataset

In [ ]:
print(merged_stocks.isnull().sum())

tda                 0
nom tda             0
prod                0
descripcion         0
inv cajas           0
CLAVE               0
AB actual           0
id                  0
Nombre de Tienda    0
dtype: int64


In [ ]:
merged_stocks[merged_stocks.isna().any(axis=1)]  #revisar registros con datos faltantes

#merged_stocks.dropna(inplace=True) #eliminar registros con datos faltantes

,tda,nom tda,prod,descripcion,inv cajas,CLAVE,AB actual,id,Nombre de Tienda


## Pivot Table

ELIMINAR los siguientes registros en el dataset

In [ ]:
sos = pd.concat([merged_sem, merged_stocks])  #unir verticalmente los datsets

In [ ]:
#eliminar los siguientes puntos de venta
condition = (sos['id'] == "T91") | \
           (sos['id'] == "T124") | \
           (sos['id'] == "G009") | \
           (sos['id'] == "G006") | \
           (sos['id'] == "G004") | \
           (sos['id'] == "G002") | \
           (sos['id'] == "C288") | \
           (sos['id'] == "C287") | \
           (sos['id'] == "C113")

# Delete rows based on the condition
sos = sos[~condition]

Remplazar el valor del punto de venta

In [ ]:
sos["id"] = sos["id"].replace(["C004"], "C354")

Construyendo la tabla pivote

In [ ]:
sos = sos.iloc[:, [0,4,6,9]]  #seleccionar columnas de interes

sos=sos[["tda", "AB actual", "cjas","inv cajas"]]  #reordenar columnas

pivot_table=pd.pivot_table(data=sos,
                           index=["tda", "AB actual",],
                           values=["cjas", "inv cajas"],
                           aggfunc=np.sum,
                           fill_value=0)    #tabla dinamica

pivot_table.to_excel("sos.xlsx")   #exportar

<ipython-input-76-1e9d75798add>:5: FutureWarning: The provided callable <function sum at 0x79b957131a80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_table=pd.pivot_table(data=sos,


#Mensual

### Venta

In [ ]:
#omitir notacion cientifica
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
mes = pd.read_csv("/content/drive/MyDrive/Sell Out/Duero/Mensual/2024/12_vta.csv", sep='|', encoding='utf8') # Explicitly use the openpyxl engine
mes.head()

,Info_del_Mes,Centro,Des_Centro,Prov,Des_Prov,Material,Des_Material,Cen.Primario,Des.Cen.Primario,Familia,Des_Familia,UMB,Vta_UMB,Vta_Cja
0,202501,C001,BODEGA ZAMORA,100998,ABSORMEX CMPC TISSUE SA DE CV,53115,HIG PREMIER RENDIPLUS 500HD 6/6ROLL,C352,CEDIS SAN ANGEL,2,ABARROTES NO COMESTIBLES,PZA,12.00,2.00
1,202501,C001,BODEGA ZAMORA,100998,ABSORMEX CMPC TISSUE SA DE CV,56015,HIG ELITE BIG ROLL 550HD+50HD 8/6ROLL,C352,CEDIS SAN ANGEL,2,ABARROTES NO COMESTIBLES,PZA,16.00,2.00
2,202501,C005,BODEGA LAZARO CARDENAS,100998,ABSORMEX CMPC TISSUE SA DE CV,28817,HIG ELITE COLOR 287HD 10/4ROLL,C002,BODEGA URUAPAN,2,ABARROTES NO COMESTIBLES,PZA,10.00,1.00
3,202501,C027,BODEGA ACAPULCO,100998,ABSORMEX CMPC TISSUE SA DE CV,23544,HIG PREMIER 400HD 12/4ROLL,NaN,NaN,2,ABARROTES NO COMESTIBLES,PZA,12.00,1.00
4,202501,C116,BODEGA LAGUNITAS,100998,ABSORMEX CMPC TISSUE SA DE CV,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,C352,CEDIS SAN ANGEL,2,ABARROTES NO COMESTIBLES,PZA,20.00,1.00


Quitar espacios a los CVS

In [ ]:
mes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2362 entries, 0 to 2361
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Info_del_Mes      2362 non-null   int64  
 1   Centro            2362 non-null   object 
 2   Des_Centro        2362 non-null   object 
 3   Prov              2362 non-null   int64  
 4   Des_Prov          2362 non-null   object 
 5   Material          2362 non-null   int64  
 6   Des_Material      2362 non-null   object 
 7   Cen.Primario      2330 non-null   object 
 8   Des.Cen.Primario  2330 non-null   object 
 9   Familia           2362 non-null   int64  
 10  Des_Familia       2362 non-null   object 
 11  UMB               2362 non-null   object 
 12  Vta_UMB           2362 non-null   float64
 13  Vta_Cja           2362 non-null   float64
dtypes: float64(2), int64(4), object(8)
memory usage: 258.5+ KB


In [ ]:
mes = mes.replace(r'\s+', ' ', regex=True) # quitar espacios en blanco, al inicio de todo proceso de limpieza

Filtrado de fechas

In [ ]:
mes=mes.loc[mes["Info_del_Mes"] != 202501]  #filtrado de fecha -1 mes actual

mes.drop(["Info_del_Mes"],axis="columns",inplace=True)  #eliminar una columna

<ipython-input-84-f2d79ecaef4b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mes.drop(["Info_del_Mes"],axis="columns",inplace=True)  #eliminar una columna


Ajustar columnas

In [ ]:
mes = mes.iloc[:, [0,1, 4,5,12]]  #seleccionar columnas por index

#Renaming columns by index (el nuevo dataset tiene nuevas columnas ordenadas)
mes.columns.values[0] = "tda"
mes.columns.values[1] = "nom tda"
mes.columns.values[2] = "prod"
mes.columns.values[3] = "descripcion"
mes.columns.values[4] = "cjas"

mes.head()

,tda,nom tda,prod,descripcion,cjas
662,C001,BODEGA ZAMORA,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,42.00
663,C001,BODEGA ZAMORA,9983,SVT PREMIER 12/450PZ,111.00
664,C001,BODEGA ZAMORA,23544,HIG PREMIER 400HD 12/4ROLL,719.00
665,C001,BODEGA ZAMORA,28817,HIG ELITE COLOR 287HD 10/4ROLL,8.00
666,C001,BODEGA ZAMORA,33798,HIG ELITE BIG ROLL 550HD 8/6PZ,1738.00


Revisar valores del dataset

In [ ]:
mes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1700 entries, 662 to 2361
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   tda          1700 non-null   object 
 1   nom tda      1700 non-null   object 
 2   prod         1700 non-null   int64  
 3   descripcion  1700 non-null   object 
 4   cjas         1700 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 79.7+ KB


In [ ]:
print("Missing values distribution: ")
print(mes.isnull().mean())
print("")
mes.shape

Missing values distribution: 
tda           0.00
nom tda       0.00
prod          0.00
descripcion   0.00
cjas          0.00
dtype: float64



(1700, 5)

Valores negativos a positivos

In [ ]:
mes['cjas'] = np.where(mes['cjas'] < 0, 0, mes['cjas'])

seleccionar valores mayores a 0´s

In [ ]:
mes = mes[(mes['cjas']>0)]

## Inventarios

In [ ]:
stockm = pd.read_csv("/content/drive/MyDrive/Sell Out/Duero/Mensual/2024/12_invt.csv", sep='|', encoding='utf8')
stockm.head()

,Info_del_Mes,Centro,Des_Centro,Prov,Des_Prov,Material,Des_Material,Marca,Des_Marca,ABC,Cen.Primario,Des.Cen.Primario,Familia,Des_Familia,Inv_Umb,Inv_Cja
0,202501,C001,BODEGA ZAMORA,100998,ABSORMEX CMPC TISSUE SA DE CV,9742,PÑ BABYSEC CLASICO CHICO 4/40PZ,3569,BABYSEC,NaN,C352,CEDIS SAN ANGEL,2,ABARROTES NO COMESTIBLES,0.00,0.00
1,202501,C001,BODEGA ZAMORA,100998,ABSORMEX CMPC TISSUE SA DE CV,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,2977,ELITE,B,C352,CEDIS SAN ANGEL,2,ABARROTES NO COMESTIBLES,560.00,28.00
2,202501,C001,BODEGA ZAMORA,100998,ABSORMEX CMPC TISSUE SA DE CV,9983,SVT PREMIER 12/450PZ,3518,PREMIER,A,C352,CEDIS SAN ANGEL,2,ABARROTES NO COMESTIBLES,768.00,64.00
3,202501,C001,BODEGA ZAMORA,100998,ABSORMEX CMPC TISSUE SA DE CV,9984,SVT PREMIER 48/100PZ,3518,PREMIER,NaN,C352,CEDIS SAN ANGEL,2,ABARROTES NO COMESTIBLES,0.00,0.00
4,202501,C001,BODEGA ZAMORA,100998,ABSORMEX CMPC TISSUE SA DE CV,11233,HIG ELITE BIG ROLL 400HD 12/4ROLL,2977,ELITE,NaN,C352,CEDIS SAN ANGEL,2,ABARROTES NO COMESTIBLES,0.00,0.00


Quitar los espacios vacios en todo el dataframe

In [ ]:
stockm = stockm.replace(r'\s+', ' ', regex=True) # quitar espacios en blanco, al inicio de todo proceso de limpieza

Delimitar el dataset

In [ ]:
stockm = stockm.iloc[:, [0,1,2,5,6,15]]  #seleccionar columnas por index

#Renaming columns by index (el nuevo dataset tiene nuevas columnas ordenadas)
stockm.columns.values[0] = "fecha"
stockm.columns.values[1] = "tda"
stockm.columns.values[2] = "nom tda"
stockm.columns.values[3] = "prod"
stockm.columns.values[4] = "descripcion"
stockm.columns.values[5] = "inv cajas"

stockm.head()

,fecha,tda,nom tda,prod,descripcion,inv cajas
0,202501,C001,BODEGA ZAMORA,9742,PÑ BABYSEC CLASICO CHICO 4/40PZ,0.00
1,202501,C001,BODEGA ZAMORA,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,28.00
2,202501,C001,BODEGA ZAMORA,9983,SVT PREMIER 12/450PZ,64.00
3,202501,C001,BODEGA ZAMORA,9984,SVT PREMIER 48/100PZ,0.00
4,202501,C001,BODEGA ZAMORA,11233,HIG ELITE BIG ROLL 400HD 12/4ROLL,0.00


Filtrar el dataset de acuerdo a la ultima fecha del inventario

In [ ]:
stockm=stockm.loc[stockm["fecha"] != 202411]  #filtrado de fecha -1mes actual

stockm.drop(["fecha"],axis="columns",inplace=True)  #eliminar una columna

valores negativos

In [ ]:
stockm['inv cajas'] = pd.to_numeric(stockm['inv cajas'], errors='coerce')  #transformar a numericos por fuerza bruta

stockm['inv cajas'] = np.where(stockm['inv cajas'] < 0, 0, stockm['inv cajas'])

valores nulos

In [ ]:
stockm = stockm[(stockm['inv cajas']>0)]

## Merged catalogos

Ventas

In [ ]:
#productos
merged_mes = mes.merge(ab, left_on="prod", right_on="CLAVE", how="left")
merged_mes.head()

,tda,nom tda,prod,descripcion,cjas,CLAVE,AB actual
0,C001,BODEGA ZAMORA,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,42.00,9979,AB10335223
1,C001,BODEGA ZAMORA,9983,SVT PREMIER 12/450PZ,111.00,9983,AB14101664
2,C001,BODEGA ZAMORA,23544,HIG PREMIER 400HD 12/4ROLL,719.00,23544,AB15090074
3,C001,BODEGA ZAMORA,28817,HIG ELITE COLOR 287HD 10/4ROLL,8.00,28817,AB12335678
4,C001,BODEGA ZAMORA,33798,HIG ELITE BIG ROLL 550HD 8/6PZ,1738.00,33798,AB17335636


In [ ]:
#tiendas
merged_mes = merged_mes.merge(tda, left_on="tda", right_on="id", how="left")
merged_mes.head()

,tda,nom tda,prod,descripcion,cjas,CLAVE,AB actual,id,Nombre de Tienda
0,C001,BODEGA ZAMORA,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,42.00,9979,AB10335223,C001,Bodega Zamora
1,C001,BODEGA ZAMORA,9983,SVT PREMIER 12/450PZ,111.00,9983,AB14101664,C001,Bodega Zamora
2,C001,BODEGA ZAMORA,23544,HIG PREMIER 400HD 12/4ROLL,719.00,23544,AB15090074,C001,Bodega Zamora
3,C001,BODEGA ZAMORA,28817,HIG ELITE COLOR 287HD 10/4ROLL,8.00,28817,AB12335678,C001,Bodega Zamora
4,C001,BODEGA ZAMORA,33798,HIG ELITE BIG ROLL 550HD 8/6PZ,1738.00,33798,AB17335636,C001,Bodega Zamora


Revisar valores nulos en el dataset

In [ ]:
print(merged_mes.isnull().sum())

tda                 0
nom tda             0
prod                0
descripcion         0
cjas                0
CLAVE               0
AB actual           0
id                  0
Nombre de Tienda    0
dtype: int64


Seleccionar los registros con valores faltantes

In [ ]:
merged_mes[merged_mes.isna().any(axis=1)]  #revisar registros con datos faltantes

#merged_mes.dropna(inplace=True) #eliminar registros con datos faltantes

,tda,nom tda,prod,descripcion,cjas,CLAVE,AB actual,id,Nombre de Tienda


Inventarios

In [ ]:
#productos
merged_stockm = stockm.merge(ab, left_on="prod", right_on="CLAVE", how="left")
merged_stockm.head()

,tda,nom tda,prod,descripcion,inv cajas,CLAVE,AB actual
0,C001,BODEGA ZAMORA,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,28.00,9979,AB10335223
1,C001,BODEGA ZAMORA,9983,SVT PREMIER 12/450PZ,64.00,9983,AB14101664
2,C001,BODEGA ZAMORA,23544,HIG PREMIER 400HD 12/4ROLL,265.00,23544,AB15090074
3,C001,BODEGA ZAMORA,28817,HIG ELITE COLOR 287HD 10/4ROLL,11.00,28817,AB12335678
4,C001,BODEGA ZAMORA,33798,HIG ELITE BIG ROLL 550HD 8/6PZ,783.38,33798,AB17335636


In [ ]:
#tiendas
merged_stockm = merged_stockm.merge(tda, left_on="tda", right_on="id", how="left")
merged_stockm.head()

,tda,nom tda,prod,descripcion,inv cajas,CLAVE,AB actual,id,Nombre de Tienda
0,C001,BODEGA ZAMORA,9979,SVTOALLA ELITE CELESTE MEGA ROLL 20PZ,28.00,9979,AB10335223,C001,Bodega Zamora
1,C001,BODEGA ZAMORA,9983,SVT PREMIER 12/450PZ,64.00,9983,AB14101664,C001,Bodega Zamora
2,C001,BODEGA ZAMORA,23544,HIG PREMIER 400HD 12/4ROLL,265.00,23544,AB15090074,C001,Bodega Zamora
3,C001,BODEGA ZAMORA,28817,HIG ELITE COLOR 287HD 10/4ROLL,11.00,28817,AB12335678,C001,Bodega Zamora
4,C001,BODEGA ZAMORA,33798,HIG ELITE BIG ROLL 550HD 8/6PZ,783.38,33798,AB17335636,C001,Bodega Zamora


Revisar valores nulos en el dataset

In [ ]:
print(merged_stockm.isnull().sum())

tda                 0
nom tda             0
prod                0
descripcion         0
inv cajas           0
CLAVE               0
AB actual           0
id                  0
Nombre de Tienda    0
dtype: int64


Seleccionar observaciones con valores nulos

In [ ]:
merged_stockm[merged_stockm.isna().any(axis=1)]  #revisar registros con datos faltantes

#merged_stockm.dropna(inplace=True) #eliminar registros con datos faltantes

,tda,nom tda,prod,descripcion,inv cajas,CLAVE,AB actual,id,Nombre de Tienda


## Pivot Table

In [ ]:
som = pd.concat([merged_mes, merged_stockm])  #unir verticalmente los datsets

ELIMINAR los siguientes puntos de venta

In [ ]:
#eliminar los siguientes puntos de venta
condition = (som['id'] == "T91") | \
           (som['id'] == "T124") | \
           (som['id'] == "G009") | \
           (som['id'] == "G006") | \
           (som['id'] == "G004") | \
           (som['id'] == "G002") | \
           (som['id'] == "C288") | \
           (som['id'] == "C287") | \
           (som['id'] == "C113")

# Delete rows based on the condition
som = som[~condition]

REMPLAZAR los siguientes puntos de venta

In [ ]:
som["id"] = som["id"].replace(["C004"], "C354")

construyendo la tabla pivote

In [ ]:
som = som.iloc[:, [4,6,7,9]]  #seleccionar columnas de interes

#som=som[["id", "AB actual", "cjas","inv cajas"]]  #reordenar columnas

# tabla pivote
pivot_table=pd.pivot_table(data=som,
                           index=["id", "AB actual",],
                           values=["cjas", "inv cajas"],
                           aggfunc= np.sum,
                           fill_value=0)

# Assuming you want the current column order
column_order = ["id", "AB actual", "cjas", "inv cajas"]

pivot_table.reset_index().to_excel('som.xlsx',sheet_name = 'Sheet1',index=False, columns= column_order)

<ipython-input-110-370007a20916>:6: FutureWarning: The provided callable <function sum at 0x79b957131a80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_table=pd.pivot_table(data=som,
